In [7]:
#Import Packages
import os
import pandas as pd
from requests import get
from bs4 import BeautifulSoup
import lxml
import time

#Import Team Data and helper functions
from constants import TEAM_TO_TEAM_ABBR, TEAM_SETS, CURRENT_TEAM
from functions import remove_accents

#Silence Warning
pd.options.mode.chained_assignment = None 

In [47]:
def get_team_stats_me(team, season_end_year, team_current):
  
  r = get(f'https://widgets.sports-reference.com/wg.fcgi?css=1&site=bbr&url=%2Fteams%2F{team}%2F{season_end_year}%2Fgamelog%2F&div=div_tgl_basic')
  

  if r.status_code==200:

    soup = BeautifulSoup(r.content, 'html.parser')
    table = soup.find('table')
    if table == None:
      df = 0 
      return df

    df = pd.read_html(str(table))[0]

    #Playoffs
    rp = get(f'https://widgets.sports-reference.com/wg.fcgi?css=1&site=bbr&url=%2Fteams%2F{team}%2F{season_end_year}%2Fgamelog%2F&div=div_tgl_basic_playoffs')
    if rp.status_code==200:
      soupp = BeautifulSoup(rp.content, 'html.parser')
      tablep = soupp.find('table')
      if tablep != None:
        df_playoffs = pd.read_html(str(tablep))[0]
        df = pd.concat([df, df_playoffs])
    

    df.columns = list(map('_'.join, df.columns.values))

    df = df.iloc[: , [2,3, 4, 5, 6, 7, 10, 27, 22, 39]]

    # print(df.columns)
    df = df.loc[df.iloc[: , 0] != 'Date']
    # df = df.dropna()

    df.columns =  ['Date', 'Venue','Opp', 'Result', 
                   'Tm_score', 'Opp_score', 'Team_FG%', 
                   'Opponent_FG%','Team_TOV','Opponent_TOV']
                   


    df['Winner'],df['Loser'], df['HomeTeam'],df['MOV'] = None, None, None, None
    df['Winner_FG%'],df['Loser_FG%'] =  None, None
    df['Winner_TOV'],df['Loser_TOV'] = None, None

    # #Fill columns of winners and losers 
    df['Winner'].loc[df.Result == 'W'], df['Loser'].loc[df.Result == 'L'] = team_current, team_current
    df['Winner'].loc[df.Result == 'L'] = df['Opp'].loc[df.Result == 'L']
    df['Loser'].loc[df.Result == 'W'] = df['Opp'].loc[df.Result == 'W']

    df['Winner_FG%'].loc[df.Result == 'W'] = df['Team_FG%'].loc[df.Result == 'W']
    df['Winner_FG%'].loc[df.Result == 'L'] = df['Opponent_FG%'].loc[df.Result == 'L']
    df['Loser_FG%'].loc[df.Result == 'L'] = df['Team_FG%'].loc[df.Result == 'L']
    df['Loser_FG%'].loc[df.Result == 'W'] = df['Opponent_FG%'].loc[df.Result == 'W']

    df['Winner_TOV'].loc[df.Result == 'W'] = df['Team_TOV'].loc[df.Result == 'W']
    df['Winner_TOV'].loc[df.Result == 'L'] = df['Opponent_TOV'].loc[df.Result == 'L']
    df['Loser_TOV'].loc[df.Result == 'L'] = df['Team_TOV'].loc[df.Result == 'L']
    df['Loser_TOV'].loc[df.Result == 'W'] = df['Opponent_TOV'].loc[df.Result == 'W']

    #Compute MOV
    df['MOV']  = abs(df[['Tm_score', 'Opp_score']].astype(float).diff(axis=1).iloc[: , -1])


    # #Compute Home team
    df['HomeTeam'].loc[df.Venue != '@'] = team_current
    df['HomeTeam'].loc[df.Venue == '@'] = df['Opp'].loc[df.Venue == '@']

    # #drop columns
    df = df.drop(axis = 1, labels = ['Opp', 'Venue',
                                     'Result', 'Tm_score', 
                                     'Opp_score','Team_FG%', 
                                     'Opponent_FG%','Team_TOV',
                                     'Opponent_TOV'])

  return df


In [37]:
#Lets do a test run
team = 'HOU'
season_end_year = 1990
gs = get_team_stats_me(team, season_end_year, team)
gs.head()

UnboundLocalError: local variable 'df' referenced before assignment

In [ ]:
#Running it over all games in whole decades
decades = [1950 + i*10 for i in range(8)]

for i in range(len(decades) - 1):
  decade = pd.DataFrame()   
  for season_end_year in range(decades[i],decades[i+1]):
    season = pd.DataFrame()   
    for (team_list, current_team) in zip(TEAM_SETS, CURRENT_TEAM):
      for team in team_list:
        time.sleep(3.1)
        df = get_team_stats_me(team, season_end_year, current_team[0])
        if isinstance(df, pd.DataFrame):
          season = pd.concat([season,df])

    season = season.drop_duplicates()
    season['Date'] = pd.to_datetime(season.Date)
    season = season.sort_values(by=['Date'])
    decade = decade.append(season)

  file_name = 'NBA_' + str(decades[i]) + '-'+ str(decades[i+1]) + '_Game-outcomes.csv'
  decade.to_csv(file_name)

In [30]:
#Running it over all games in whole decades
current_decade = [2020,2021,2022]
decade = pd.DataFrame()   
for season_end_year in current_decade:
    print(season_end_year)
    season = pd.DataFrame()   
    for (team_list, current_team) in zip(TEAM_SETS, CURRENT_TEAM):
        for team in team_list:
            df = get_team_stats_me(team, season_end_year, current_team[0])
            if isinstance(df, pd.DataFrame):
                season = pd.concat([season,df])

    season = season.drop_duplicates()
    season['Date'] = pd.to_datetime(season.Date)
    season = season.sort_values(by=['Date'])
    decade = decade.append(season)

# file_name = '../data/raw/NBA_' + str(current_decade[0]) + '-'+ str(current_decade[-1]) + '_Game-outcomes.csv'
# decade.to_csv(file_name)

2020


UnboundLocalError: local variable 'df' referenced before assignment

In [ ]:
file_name = '../data/raw/NBA_' + str(current_decade[0]) + '-'+ str(current_decade[-1]) + '_Game-outcomes.csv'
decade.to_csv(file_name)